### 3. 仮説検定の種類

#### 3.1_ 1変量データのt検定

In [23]:
import numpy as np
import pandas as pd
import scipy as sp
from scipy import stats

import matplotlib as mpl
from matplotlib import pyplot as plt
import seaborn as sns
sns.set()

%matplotlib inline

In [1]:
#csvの読み込み
data1 = pd.read_csv('Lesson5-1.csv')

NameError: name 'pd' is not defined

In [25]:
#標本平均の計算(=母平均の点推定)
mu = sp.mean(data1)
mu

Length    51.820183
dtype: float64

In [26]:
#自由度(標本分散⇒不偏分散への補正。n-1の決まり)
df = len(data1) - 1
df

29

In [27]:
#標準誤差の計算
sigma = sp.std(data1, ddof=1)
se = sigma/sp.sqrt(len(data1))
se

#ddofはデータの個数nから引く値。
#誤差(偏差)計算時の自由度反映。

Length    0.562163
dtype: float64

この情報を元に「池の魚平均体長は50cmと等しいか」について調べてみましょう。両側検定になりますね。有意水準は5％とします。帰無仮説・対立仮説は以下の通りです。

帰無仮説（H0）: 池の中の魚の平均体長は50cmである
対立仮説（H1）: 池の中の魚の平均体長は50cmとは異なる =50cmより大きいor50cmより小さい

In [28]:
#t値の計算
t_value = (mu-50)/se
t_value

Length    3.237819
dtype: float64

In [29]:
#p値の計算
alpha = stats.t.cdf(t_value, df=df)
(1-alpha)*2

#(1-alpha)は上側信頼限界の外の面積（確率）。なので左右対称の下側分も求めるため*2する

array([0.00301227])

p値：帰無仮説が正しいという仮定のもとで、標本から計算した検定統計量の実現値以上の値が得られる確率。p値が有意水準より小さい時に帰無仮説を棄却する
[参考: p値が小さいことの意味]
p値の大きさが、対立仮説を採択＝帰無仮説を棄却する決め手となります。
p値が小さいということは、『帰無仮説が正しいとすると』確率的にほとんど起こりえないことが起きたということを意味します。(有意水準が5%なら100回中5回以下、 1%なら100回中1回以下)
逆にp値が大きいということは、確率的にはよくあることが起きた(だから、この結果では差があるとはいえない)、ということになります。

In [30]:
#尚、以上の計算はstats.ttest_1samp関数を用いると下記のように実行できます。
stats.ttest_1samp(data1, 50)

#標本データdata1をもとに、
#帰無仮説「池の中の魚の平均体長は50cmである」を両側検定

Ttest_1sampResult(statistic=array([3.23781919]), pvalue=array([0.00301227]))

次に、「池の魚の平均体長は50cmより大きい」かどうかを調べてみましょう。
こちらも有意差5％とします。今度は片側検定を実行することになります。仮説は以下の通りです。
 帰無仮説（H0）: 池の中の魚の平均体長は50cmである
 対立仮説（H1）: 池の中の魚の平均体長は50cmより大きい


In [31]:
#t値の計算までは両側検定と一緒です。片側検定では分布の片側だけに棄却域を取りました。
#ですので、p値の計算は以下のようになります。
alpha = stats.t.cdf(t_value, df = df)
1-alpha

array([0.00150613])

#### 3.2_  2変量データのt検定

今までは1つの池の中の魚の体長といった「1変量のデータ」について考えてきました。次は、「飼料Aで育った養殖魚と飼料Bで育った養殖魚の体長」「2歳魚と3歳魚の体長」といった2変数の間で平均値に差があるか否かについて考えてみましょう。

まずは「対応のある」t検定

In [32]:
data2 = pd.read_csv('Lesson5-2.csv')

In [33]:
data2.head()

,2-years-old,3-years-old
0,16.624345,14.600841
1,14.388244,14.984220
2,14.471828,14.606675
3,13.927031,14.401233
4,15.865408,14.627380


In [35]:
#それぞれの魚について2歳時点での体長と3歳時点での体長のデータが記入されているとします。
#次に、この2変数の差を計算してみます。
data2['Difference']= data2['3-years-old']- data2['2-years-old']
data2.head()

,2-years-old,3-years-old,Difference
0,16.624345,14.600841,-2.023504
1,14.388244,14.984220,0.595977
2,14.471828,14.606675,0.134847
3,13.927031,14.401233,0.474201
4,15.865408,14.627380,-1.238028


対応のあるt検定では「差が0と異なるか」という1変数のt検定を行うことになります。

帰無仮説と対立仮説について確認しましょう。対応のあるt検定では下記の仮説を立てることになりますが、これは括弧書きのように解釈できるという意味です。

  帰無仮説（H0）: 2歳魚と3歳魚で体長は変わらない（2歳魚と3歳魚の体長の差は0）  
  対立仮説（H1）: 2歳魚と3歳魚で体長は異なる（2歳魚と3歳魚の体長の差は0ではない）

In [36]:
stats.ttest_1samp(data2['Difference'], 0)

Ttest_1sampResult(statistic=2.212896760339997, pvalue=0.03493196300425262)

p値が0.05を下回りましたので、「2歳魚と3歳魚の体長は有意に異なる」と判断できました

### 「対応のない」t検定

対応のあるt検定では、それぞれの個体について2歳時点と3歳時点の2回計測を行った結果について解析しました。  
それでは、このデータが「2歳魚を集めたデータ」と「3歳魚を集めたデータ」ではあるものの同じ個体の2歳時と3歳時の体長を測定したものではないとするとどうなるでしょうか。今度は「平均値の差」について注目する必要があります。

平均値の差に基づいてt値を計算する場合、少し計算式が複雑になります。2歳魚の体長をX、3歳魚の体長をYとすると下記のように計算できます。
（省略）

対応のないt検定はstats.ttest_ind関数を用いると簡単に計算できます

[ttest_indを使う場合]  
対応がないt検定。  
２つの集団からの各対象から、１つずつ値を抜き出してきて、平均値の差が有意かどうかを調べる検定。  
    res = stats.ttest_ind(group1, group2, equal_var = False)  
2つの集団の分散が異なる場合には、equal_var = Falseオプションを付ける。

In [37]:
stats.ttest_ind(data2['2-years-old'], data2['3-years-old'], equal_var= False)

Ttest_indResult(statistic=-2.403807649264687, pvalue=0.01945646990570736)